## Step 1: Setup and Data Loading

In [1]:
import pandas as pd
import numpy as np
from collections import defaultdict, Counter
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

# Set display options for better readability
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 100)

print("Libraries imported successfully")

Libraries imported successfully


In [2]:
# Load CSV files, skipping the first 6 lines of metadata
file_paths = {
    2021: './data/2021-total-cost-keywords.csv',
    2022: './data/2022-total-cost-keywords.csv', 
    2023: './data/2023-total-cost-keywords.csv',
    2024: './data/2024-total-cost-keywords.csv',
    2025: './data/2025-total-cost-keywords.csv'
}

# Load all dataframes
dataframes = {}
for year, path in file_paths.items():
    print(f"Loading {year} data...")
    df = pd.read_csv(path, skiprows=6)  # Skip first 6 lines of metadata
    print(f"  - Raw shape: {df.shape}")
    dataframes[year] = df
    
print("\nAll files loaded successfully!")

Loading 2021 data...
  - Raw shape: (11811, 4)
Loading 2022 data...
  - Raw shape: (11921, 4)
Loading 2023 data...
  - Raw shape: (11647, 4)
Loading 2024 data...
  - Raw shape: (10603, 4)
Loading 2025 data...
  - Raw shape: (8493, 4)

All files loaded successfully!


## Step 2: Data Preprocessing

In [3]:
# Examine the structure of one dataframe
print("Sample dataframe structure (2021):")
print(f"Columns: {list(dataframes[2021].columns)}")
print(f"\nFirst few rows:")
print(dataframes[2021].head())
print(f"\nData types:")
print(dataframes[2021].dtypes)

Sample dataframe structure (2021):
Columns: ['Project Terms', 'Total Cost', 'Total Cost(Sub Projects)', 'Unnamed: 3']

First few rows:
                                                                                         Project Terms  \
0                                                                                                        
1                                                                                                        
2  Accounting;Address;Adult;Adverse effects;Adverse event;Age;Alcohol consumption;Alcohols;American...   
3  Affect;African American;Area;Attitude;Big Data;Biological;Biological Markers;Biological Process;...   
4  Activities of Daily Living;Address;Affect;Age;Aging;Anxiety;Biological;Biological Aging;Body mas...   

  Total Cost Total Cost(Sub Projects)  Unnamed: 3  
0     119029                                  NaN  
1     111964                                  NaN  
2     693883                                  NaN  
3     374998        

In [4]:
def preprocess_dataframe(df, year):
    """
    Clean and preprocess the dataframe:
    1. Drop unnecessary columns
    2. Remove rows with empty Project Terms or Total Cost
    3. Clean and validate data types
    """
    print(f"Preprocessing {year} data...")
    
    # Make a copy to avoid modifying original
    df_clean = df.copy()
    
    # Drop unnecessary columns
    columns_to_drop = []
    if 'Total Cost(Sub Projects)' in df_clean.columns:
        columns_to_drop.append('Total Cost(Sub Projects)')
    if 'Unnamed: 3' in df_clean.columns:
        columns_to_drop.append('Unnamed: 3')
    
    if columns_to_drop:
        df_clean = df_clean.drop(columns=columns_to_drop)
        print(f"  - Dropped columns: {columns_to_drop}")
    
    # Remove rows where Project Terms or Total Cost are empty/null
    initial_rows = len(df_clean)
    
    # Remove rows with empty or whitespace-only Project Terms
    df_clean = df_clean[df_clean['Project Terms'].notna()]
    df_clean = df_clean[df_clean['Project Terms'].str.strip() != '']
    
    # Remove rows with empty or null Total Cost
    df_clean = df_clean[df_clean['Total Cost'].notna()]
    
    # Convert Total Cost to numeric, handling any string formatting
    df_clean['Total Cost'] = pd.to_numeric(df_clean['Total Cost'], errors='coerce')
    df_clean = df_clean[df_clean['Total Cost'].notna()]
    
    final_rows = len(df_clean)
    rows_removed = initial_rows - final_rows
    
    print(f"  - Initial rows: {initial_rows}")
    print(f"  - Final rows: {final_rows}")
    print(f"  - Rows removed: {rows_removed}")
    print(f"  - Total funding: ${df_clean['Total Cost'].sum():,.0f}")
    
    return df_clean

# Preprocess all dataframes
clean_dataframes = {}
for year, df in dataframes.items():
    clean_dataframes[year] = preprocess_dataframe(df, year)
    print()

print("Data preprocessing completed!")

Preprocessing 2021 data...
  - Dropped columns: ['Total Cost(Sub Projects)', 'Unnamed: 3']
  - Initial rows: 11811
  - Final rows: 11279
  - Rows removed: 532
  - Total funding: $6,121,521,522

Preprocessing 2022 data...
  - Dropped columns: ['Total Cost(Sub Projects)', 'Unnamed: 3']
  - Initial rows: 11921
  - Final rows: 11402
  - Rows removed: 519
  - Total funding: $7,449,682,187

Preprocessing 2023 data...
  - Dropped columns: ['Total Cost(Sub Projects)', 'Unnamed: 3']
  - Initial rows: 11647
  - Final rows: 11126
  - Rows removed: 521
  - Total funding: $6,506,773,772

Preprocessing 2024 data...
  - Dropped columns: ['Total Cost(Sub Projects)', 'Unnamed: 3']
  - Initial rows: 10603
  - Final rows: 10194
  - Rows removed: 409
  - Total funding: $6,166,728,801

Preprocessing 2025 data...
  - Dropped columns: ['Total Cost(Sub Projects)', 'Unnamed: 3']
  - Initial rows: 8493
  - Final rows: 8171
  - Rows removed: 322
  - Total funding: $5,819,319,619

Data preprocessing completed!


## Step 3: Keyword Extraction and Normalization

In [ ]:
def extract_keywords_from_dataframe(df, year):
    """
    Extract all unique keywords from a dataframe's Project Terms column.
    Keywords are separated by semicolons and need to be normalized.
    Also adds a 'keywords_list' column to the dataframe for efficient lookup.
    """
    print(f"Extracting keywords from {year} data...")
    
    all_keywords = set()
    keywords_lists = []
    
    for project_terms in df['Project Terms']:
        if pd.notna(project_terms):
            # Split on semicolon and normalize
            keywords = [kw.strip().lower() for kw in str(project_terms).split(';')]
            # Remove empty keywords
            keywords = [kw for kw in keywords if kw and kw != '']
            all_keywords.update(keywords)
            keywords_lists.append(keywords)
        else:
            keywords_lists.append([])
    
    # Add the keywords list as a new column
    df['keywords_list'] = keywords_lists
    
    print(f"  - Found {len(all_keywords)} unique keywords")
    print(f"  - Added keywords_list column to dataframe")
    return all_keywords

In [6]:
clean_dataframes.keys()

dict_keys([2021, 2022, 2023, 2024, 2025])

In [7]:
# Extract keywords from each year
yearly_keywords = {}
for year, df in clean_dataframes.items():
    yearly_keywords[year] = extract_keywords_from_dataframe(df, year)

Extracting keywords from 2021 data...
  - Found 28914 unique keywords
Extracting keywords from 2022 data...
  - Found 29244 unique keywords
Extracting keywords from 2023 data...
  - Found 29677 unique keywords
Extracting keywords from 2024 data...
  - Found 29390 unique keywords
Extracting keywords from 2025 data...
  - Found 28094 unique keywords


In [8]:
# Print sample keywords from 2021
list(yearly_keywords.get(2021))[:10]

['fossil fuels',
 'epithelial stem cell',
 'organic contaminant',
 'rain',
 'child protective service',
 'acceptability and feasibility',
 'community center',
 'chronic abdominal pain',
 'biosignature',
 'standardized care']

In [9]:

# Create master list of all unique keywords
all_unique_keywords = set()
for keywords in yearly_keywords.values():
    all_unique_keywords.update(keywords)

print(f"\nTotal unique keywords across all years: {len(all_unique_keywords)}")

# Display some sample keywords
sample_keywords = list(all_unique_keywords)[:10]
print(f"\nSample keywords: {sample_keywords}")


Total unique keywords across all years: 41742

Sample keywords: ['fossil fuels', 'epithelial stem cell', 'organic contaminant', 'rain', 'child protective service', 'acceptability and feasibility', 'community center', 'chronic abdominal pain', 'biosignature', 'standardized care']


## Step 4: Calculate Annual Keyword Metrics

In [ ]:
def calculate_keyword_metrics_for_year(df, year, all_keywords):
    """
    For each keyword, calculate:
    - Number of grants containing the keyword
    - Total funding for grants containing the keyword
    """
    print(f"Calculating metrics for {year}...")
    
    keyword_metrics = {}
    total_grants = len(df)
    total_funding = df['Total Cost'].sum()
    
    print(f"  - Processing {len(all_keywords)} keywords for {total_grants} grants")
    
    # Use tqdm for progress bar on large keyword sets
    for keyword in tqdm(all_keywords, desc=f"Processing {year}"):
        # Find grants that contain this keyword using list membership
        mask = df['keywords_list'].apply(lambda kw_list: keyword in kw_list)
        grants_with_keyword = df[mask]
        
        grant_count = len(grants_with_keyword)
        funding_total = grants_with_keyword['Total Cost'].sum() if grant_count > 0 else 0
        
        keyword_metrics[keyword] = {
            'grant_count': grant_count,
            'funding_total': funding_total,
            'grant_percentage': (grant_count / total_grants) * 100,
            'funding_proportion': (funding_total / total_funding) * 100
        }
    
    return keyword_metrics, total_grants, total_funding

In [ ]:

# Calculate metrics for each year
annual_metrics = {}
annual_totals = {}

for year, df in clean_dataframes.items():
    metrics, total_grants, total_funding = calculate_keyword_metrics_for_year(
        df, year, all_unique_keywords
    )
    annual_metrics[year] = metrics
    annual_totals[year] = {
        'total_grants': total_grants,
        'total_funding': total_funding
    }
    print(f"  - Completed {year}: {total_grants} grants, ${total_funding:,.0f} total funding\n")

print("Annual metrics calculation completed!")

## Step 5: Compute Proportional Comparisons

In [ ]:
# Calculate 2021-2024 baseline metrics
print("Calculating 2021-2024 baseline metrics...")

baseline_years = [2021, 2022, 2023, 2024]
baseline_metrics = {}

# Calculate total baseline grants and funding
baseline_total_grants = sum(annual_totals[year]['total_grants'] for year in baseline_years)
baseline_total_funding = sum(annual_totals[year]['total_funding'] for year in baseline_years)

print(f"Baseline period (2021-2024):")
print(f"  - Total grants: {baseline_total_grants:,}")
print(f"  - Total funding: ${baseline_total_funding:,.0f}")

# Aggregate baseline metrics for each keyword
for keyword in tqdm(all_unique_keywords, desc="Calculating baseline metrics"):
    baseline_grant_count = sum(annual_metrics[year][keyword]['grant_count'] for year in baseline_years)
    baseline_funding_total = sum(annual_metrics[year][keyword]['funding_total'] for year in baseline_years)
    
    baseline_metrics[keyword] = {
        'grant_count': baseline_grant_count,
        'funding_total': baseline_funding_total,
        'grant_percentage': (baseline_grant_count / baseline_total_grants) * 100,
        'funding_proportion': (baseline_funding_total / baseline_total_funding) * 100
    }

print("\n2025 metrics:")
print(f"  - Total grants: {annual_totals[2025]['total_grants']:,}")
print(f"  - Total funding: ${annual_totals[2025]['total_funding']:,.0f}")

print("\nBaseline calculations completed!")

## Step 6: Compile Final Results

In [ ]:
# Create final comparison dataframe
print("Compiling final results...")

results_data = []

for keyword in all_unique_keywords:
    # Get 2025 metrics
    metrics_2025 = annual_metrics[2025][keyword]
    metrics_baseline = baseline_metrics[keyword]
    
    # Calculate changes
    funding_prop_change = metrics_2025['funding_proportion'] - metrics_baseline['funding_proportion']
    grant_pct_change = metrics_2025['grant_percentage'] - metrics_baseline['grant_percentage']
    
    results_data.append({
        'keyword': keyword,
        'funding_proportion_2025': metrics_2025['funding_proportion'],
        'grant_percentage_2025': metrics_2025['grant_percentage'],
        'funding_proportion_2021_2024': metrics_baseline['funding_proportion'],
        'grant_percentage_2021_2024': metrics_baseline['grant_percentage'],
        'funding_proportion_change': funding_prop_change,
        'grant_percentage_change': grant_pct_change,
        'grants_2025': metrics_2025['grant_count'],
        'grants_baseline': metrics_baseline['grant_count'],
        'funding_2025': metrics_2025['funding_total'],
        'funding_baseline': metrics_baseline['funding_total']
    })

# Create DataFrame and sort by funding impact
results_df = pd.DataFrame(results_data)

print(f"Results compiled for {len(results_df)} keywords")
print(f"Results dataframe shape: {results_df.shape}")

In [ ]:
# Display summary statistics
print("=== SUMMARY STATISTICS ===")
print(f"\nTotal unique keywords analyzed: {len(results_df):,}")
print(f"\n2025 vs 2021-2024 Comparison:")
print(f"  - 2025 total grants: {annual_totals[2025]['total_grants']:,}")
print(f"  - 2021-2024 total grants: {baseline_total_grants:,}")
print(f"  - 2025 total funding: ${annual_totals[2025]['total_funding']:,.0f}")
print(f"  - 2021-2024 total funding: ${baseline_total_funding:,.0f}")

print(f"\nKeywords with highest funding in 2025:")
top_2025_funding = results_df.nlargest(10, 'funding_proportion_2025')[['keyword', 'funding_proportion_2025', 'grant_percentage_2025']]
print(top_2025_funding.to_string(index=False))

print(f"\nKeywords with largest funding proportion increases:")
top_increases = results_df.nlargest(10, 'funding_proportion_change')[['keyword', 'funding_proportion_change', 'funding_proportion_2025', 'funding_proportion_2021_2024']]
print(top_increases.to_string(index=False))

print(f"\nKeywords with largest funding proportion decreases:")
top_decreases = results_df.nsmallest(10, 'funding_proportion_change')[['keyword', 'funding_proportion_change', 'funding_proportion_2025', 'funding_proportion_2021_2024']]
print(top_decreases.to_string(index=False))

In [ ]:
# Filter for keywords with meaningful presence (appear in at least 10 grants or $1M+ funding)
significant_keywords = results_df[
    (results_df['grants_2025'] >= 10) | 
    (results_df['grants_baseline'] >= 10) |
    (results_df['funding_2025'] >= 1_000_000) |
    (results_df['funding_baseline'] >= 1_000_000)
].copy()

print(f"\n=== SIGNIFICANT KEYWORDS ANALYSIS ===")
print(f"Keywords meeting significance criteria: {len(significant_keywords):,}")
print(f"(At least 10 grants in any period OR $1M+ funding in any period)")

print(f"\nTop 20 keywords by 2025 funding proportion:")
top_significant = significant_keywords.nlargest(20, 'funding_proportion_2025')
display_cols = ['keyword', 'funding_proportion_2025', 'funding_proportion_2021_2024', 
                'funding_proportion_change', 'grants_2025']
print(top_significant[display_cols].to_string(index=False))

In [ ]:
# Save results to CSV files
print("Saving results to CSV files...")

# Save complete results
results_df.to_csv('nih_keyword_analysis_complete.csv', index=False)
print(f"  - Complete results saved: nih_keyword_analysis_complete.csv ({len(results_df)} rows)")

# Save significant keywords only
significant_keywords.to_csv('nih_keyword_analysis_significant.csv', index=False)
print(f"  - Significant keywords saved: nih_keyword_analysis_significant.csv ({len(significant_keywords)} rows)")

# Save top changers for easy review
top_changers = pd.concat([
    significant_keywords.nlargest(50, 'funding_proportion_change'),
    significant_keywords.nsmallest(50, 'funding_proportion_change')
]).drop_duplicates()

top_changers.to_csv('nih_keyword_analysis_top_changes.csv', index=False)
print(f"  - Top changes saved: nih_keyword_analysis_top_changes.csv ({len(top_changers)} rows)")

print("\nAnalysis complete! 🎉")